# Tarefa 3: utilizar o Amazon Bedrock para responder perguntas

Neste caderno, você aprenderá a usar o modelo Bedrock Nova Lite para fornecer respostas informativas a consultas, enviando a solicitação com todo o contexto relevante ao modelo e aguardando a resposta, enfrentando o desafio de obter respostas factuais para perguntas sem a necessidade de preparar e indexar documentos previamente.

Este caderno simula o que a **Geração aumentada de recuperação (RAG)** faria, mas sem usar RAG de fato. Essa abordagem funciona para documentos curtos ou aplicações singleton. Sua escala pode não ser adequada para responder perguntas em nível empresarial, visto que nem todos os documentos grandes necessários cabem no prompt enviado para o modelo.

**Responder perguntas (QA)** é uma tarefa importante que envolve extrair respostas para consultas factuais feitas em linguagem natural. Normalmente, um sistema de QA processa uma consulta em uma base de conhecimento que contém dados estruturados ou não estruturados e gera uma resposta com informações precisas. Garantir um alto nível de acurácia é essencial para desenvolver um sistema de resposta a perguntas útil e confiável, principalmente para casos de uso empresariais.


## Cenário

Você tenta modelar uma situação na AnyCompany em que pede para que um modelo de respostas a perguntas dê informações sobre a troca de pneus para um modelo de veículo específico que ela fabrica. Primeiro, você consulta o modelo usando uma abordagem "zero-shot" para ver se ele pode dar respostas relevantes com base apenas os dados de treinamento dele.

No entanto, você percebe que o modelo parece estar "alucinando" respostas mais genéricas, conforme evidenciado quando você testa um modelo de veículo falso e obtém respostas semelhantes. Isso sugere a necessidade de aumentar o treinamento do modelo com os manuais de veículos reais da Example Company para dar detalhes sobre os pneus de cada modelo.

Neste laboratório, você simula essa abordagem de "geração aumentada de recuperação" (RAG) sem dados externos. Você fornece um trecho detalhado do manual explicando como trocar os pneus no veículo Modelo Z da AnyCompany. Você testa se o modelo agora pode fornecer uma resposta personalizada e precisa usando esse exemplo de conteúdo contextualizado.

## Tarefa 3.1: Configurar o ambiente

Nesta tarefa, você configurará o ambiente.

In [ ]:
#ignore warnings and create a service client by name using the default session.
import json
import os
import sys
import warnings

import boto3
import botocore

warnings.filterwarnings('ignore')
module_path = ".."
sys.path.append(os.path.abspath(module_path))
bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", None))



In [ ]:
"""
Nova Lite Adapter:

This code block contains helper functions for using Nova Lite.
"""

import json
import time
from botocore.exceptions import ClientError

def format_for_nova_lite(prompt_text):
    """Format the prompt for Nova Lite's expected message structure."""
    return {
        "messages": [
            {
                "role": "user",
                "content": [{"text": prompt_text}]
            }
        ],
        "inferenceConfig": {
            "maxTokens": 2048,
            "temperature": 0,
            "topP": 0.9
        }
    }

def parse_nova_lite_response(response_body):
    """Parse the response from Nova Lite."""
    if 'output' in response_body and 'message' in response_body['output']:
        message = response_body['output']['message']
        if 'content' in message and isinstance(message['content'], list):
            # Extract text from each content item
            texts = []
            for content_item in message['content']:
                if isinstance(content_item, dict) and 'text' in content_item:
                    texts.append(content_item['text'])
            return ' '.join(texts)
    
    # Fallback if the response format is different
    return str(response_body)

## Tarefa 3.2: Perguntas e respostas com o conhecimento do modelo
Nesta seção, tentamos usar um modelo oferecido pelo serviço Bedrock para responder a perguntas com base no conhecimento que ele adquiriu durante a fase de treinamento.

Nesta tarefa, você usará o método invoke_model () do cliente do Amazon Bedrock. Os parâmetros obrigatórios para usar esse método são: "modelId", que representa o ARN do modelo do Amazon Bedrock; além de "body", que é o prompt de sua tarefa.

O formato do texto do prompt muda conforme o provedor do modelo de base selecionado. Você verá isso em detalhes abaixo.

```json
{
   modelId= model_id,
   contentType= "application/json",
   accept= "application/json",
   body=body
}

```

Você tenta usar modelos fornecidos pelo serviço Bedrock para responder perguntas com base no conhecimento adquirido durante a fase de treinamento.

In [ ]:
prompt_data = """You are an helpful automotive assistant. Answer questions in a concise way. When answering questions about car maintenance or repairs, provide a detailed, numbered list of steps. Assume all cars, including the AnyCompany AC8, have a spare tire. If you are unsure about the
answer say 'I am unsure'

Question: How can I fix a flat tire on my AnyCompany AC8?
Answer:"""


<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **Nota:** ao executar o código em tarefas futuras, você pode observar mensagens de *ThrottlingException* e novas tentativas. Seu código inclui um tratamento robusto de erros que repetirá automaticamente as solicitações que falharem com recuo exponencial. Esse é um comportamento normal ao trabalhar com cotas de serviço e demonstra como os aplicativos prontos para produção devem lidar com as limitações da API.

## Tarefa 3.3: Invocar o modelo passando o formato de texto JSON para gerar a resposta

In [ ]:
import json
import time
from botocore.exceptions import ClientError
# Model configuration
modelId = "amazon.nova-lite-v1:0"
accept = "application/json"
contentType = "application/json"

# Retry configuration
max_retries = 5
retry_delay = 10  # seconds

def invoke_model_with_retry(bedrock_client, modelId, prompt_text, contentType, accept, max_retries, retry_delay):
    """Invoke the model with retry logic."""
    # Format the prompt for Nova Lite
    body = format_for_nova_lite(prompt_text)
    
    for attempt in range(max_retries):
        try:
            response = bedrock_client.invoke_model(
                modelId=modelId,
                body=json.dumps(body),
                contentType=contentType,
                accept=accept
            )
            
            response_body = json.loads(response.get('body').read())
            return parse_nova_lite_response(response_body)

        except ClientError as error:
            if error.response['Error']['Code'] == 'AccessDeniedException':
                print(f"\x1b[41m{error.response['Error']['Message']}\\n\
                \nTo troubleshoot this issue please refer to the following resources:\\n\
                \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\\n\
                \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")      
                raise

            elif error.response['Error']['Code'] in ['ThrottlingException', 'ServiceUnavailableException']:
                if attempt < max_retries - 1:
                    print(f"Service capacity reached. Retrying in {retry_delay} seconds...")
                    time.sleep(retry_delay)
                    retry_delay *= 2  # Exponential backoff
                else:
                    print("Max retries reached. Unable to invoke the model.")
                    raise

            else:
                print(f"An error occurred: {error}")
                raise

        except Exception as e:
            print(f"An unexpected error occurred: {e}")
            raise

# Main execution
try:
    result = invoke_model_with_retry(bedrock_client, modelId, prompt_data, contentType, accept, max_retries, retry_delay)
    print(result)
except Exception as e:
    print(f"Failed to invoke the model after retries: {e}")




O modelo fornece uma resposta descrevendo o processo de troca do pneu furado do carro, mas a mesma explicação pode ser válida para qualquer carro. Infelizmente, essa não é a resposta correta para o modelo AC8 da AnyCompany, que não tem estepe. Isso ocorre porque o modelo foi treinado com dados que contêm instruções sobre a troca de pneus em carros.

Outro exemplo desse problema pode ser encontrado ao tentar fazer a mesma pergunta usando uma marca e modelo de carro totalmente falsos, como Amazon Tirana.

In [ ]:
import json
import time
from botocore.exceptions import ClientError

# Usage
prompt_data = """How can I fix a flat tire on my Amazon Tirana?
Answer: Here are the steps to fix a flat tire:
"""

def invoke_model_with_retry(prompt_data, max_retries=5, initial_delay=10):
    modelId = "amazon.nova-lite-v1:0"
    accept = "application/json"
    contentType = "application/json"
    
    # Format the prompt for Nova Lite
    body = json.dumps(format_for_nova_lite(prompt_data))
    
    for attempt in range(max_retries):
        try:
            response = bedrock_client.invoke_model(
                body=body, 
                modelId=modelId, 
                accept=accept, 
                contentType=contentType
            )
            response_body = json.loads(response.get("body").read())
            return parse_nova_lite_response(response_body)

        except ClientError as error:
            error_code = error.response['Error']['Code']
            
            if error_code == 'AccessDeniedException':
                print(f"\x1b[41m{error.response['Error']['Message']}\\n\
                \nTo troubleshoot this issue please refer to the following resources:\\n\
                \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\\n\
                \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
                raise
                
            elif error_code in ['ThrottlingException', 'ServiceUnavailableException', 'ModelStreamLimitExceededException']:
                if attempt < max_retries - 1:
                    delay = initial_delay * (2 ** attempt)  # Exponential backoff
                    print(f"Service capacity reached. Retrying in {delay} seconds...")
                    print(f"Error: {error}")
                    time.sleep(delay)
                    continue
                else:
                    print(f"Max retries ({max_retries}) reached. Last error: {error}")
                    raise
            else:
                print(f"Unhandled error occurred: {error}")
                raise
                
        except Exception as e:
            print(f"Unexpected error: {e}")
            if attempt < max_retries - 1:
                delay = initial_delay * (2 ** attempt)
                print(f"Retrying in {delay} seconds...")
                time.sleep(delay)
                continue
            else:
                raise

try:
    # Invoke model with retry logic
    result = invoke_model_with_retry(prompt_data)
    # Print the raw result
    if result:
        print(result.strip())
    else:
        print("No response generated from the model.")

except Exception as e:
    print(f"Failed to get response after all retries: {e}")


Considerando a pergunta do prompt, o modelo é incapaz de fornecer uma resposta realista.

Para resolver esse problema e fazer com que o modelo dê respostas com base nas instruções específicas válidas para o modelo do seu carro, você pode aumentar imediatamente o conhecimento do modelo fornecendo uma base de conhecimento adicional como parte do prompt.

Vamos ver como você pode usar isso para melhorar sua aplicação.

Suponha que o trecho a seguir seja do manual do AnyCompany AC8 (na realidade, não é o manual real, mas trate-o como tal). Este documento também é curto o suficiente para ser totalmente incluído na janela de contexto do Nova Lite.

```plain
Tires and Tire Pressure:

Tires are made of black rubber and are mounted on the wheels of your vehicle. They provide the necessary grip for driving, cornering, and braking. Two important factors to consider are tire pressure and tire wear, as they can affect the performance and handling of your car.

Where to Find Recommended Tire Pressure:

You can find the recommended tire pressure specifications on the inflation label located on the driver's side B-pillar of your vehicle. Alternatively, you can refer to your vehicle's manual for this information. The recommended tire pressure may vary depending on the speed and the number of occupants or maximum load in the vehicle.

Reinflating the Tires:

When checking tire pressure, it is important to do so when the tires are cold. This means allowing the vehicle to sit for at least three hours to ensure the tires are at the same temperature as the ambient temperature.

To reinflate the tires:

    Check the recommended tire pressure for your vehicle.
    Follow the instructions provided on the air pump and inflate the tire(s) to the correct pressure.
    In the center display of your vehicle, open the "Car status" app.
    Navigate to the "Tire pressure" tab.
    Press the "Calibrate pressure" option and confirm the action.
    Drive the car for a few minutes at a speed above 30 km/h to calibrate the tire pressure.

Note: In some cases, it may be necessary to drive for more than 15 minutes to clear any warning symbols or messages related to tire pressure. If the warnings persist, allow the tires to cool down and repeat the above steps.

Flat Tire:

If you encounter a flat tire while driving, you can temporarily seal the puncture and reinflate the tire using a tire mobility kit. This kit is typically stored under the lining of the luggage area in your vehicle.

Instructions for Using the Tire Mobility Kit:

    Open the tailgate or trunk of your vehicle.
    Lift up the lining of the luggage area to access the tire mobility kit.
    Follow the instructions provided with the tire mobility kit to seal the puncture in the tire.
    After using the kit, make sure to securely put it back in its original location.
    Contact Rivesla or an appropriate service for assistance with disposing of and replacing the used sealant bottle.

Please note that the tire mobility kit is a temporary solution and is designed to allow you to drive for a maximum of 10 minutes or 8 km (whichever comes first) at a maximum speed of 80 km/h. It is advisable to replace the punctured tire or have it repaired by a professional as soon as possible.
```

In [ ]:
context = """Tires and tire pressure:

Tires are made of black rubber and are mounted on the wheels of your vehicle. They provide the necessary grip for driving, cornering, and braking. Two important factors to consider are tire pressure and tire wear, as they can affect the performance and handling of your car.

Where to find recommended tire pressure:

You can find the recommended tire pressure specifications on the inflation label located on the driver's side B-pillar of your vehicle. Alternatively, you can refer to your vehicle's manual for this information. The recommended tire pressure may vary depending on the speed and the number of occupants or maximum load in the vehicle.

Reinflating the tires:

When checking tire pressure, it is important to do so when the tires are cold. This means allowing the vehicle to sit for at least three hours to ensure the tires are at the same temperature as the ambient temperature.

To reinflate the tires:

    Check the recommended tire pressure for your vehicle.
    Follow the instructions provided on the air pump and inflate the tire(s) to the correct pressure.
    In the center display of your vehicle, open the "Car status" app.
    Navigate to the "Tire pressure" tab.
    Press the "Calibrate pressure" option and confirm the action.
    Drive the car for a few minutes at a speed above 30 km/h to calibrate the tire pressure.

Note: In some cases, it may be necessary to drive for more than 15 minutes to clear any warning symbols or messages related to tire pressure. If the warnings persist, allow the tires to cool down and repeat the above steps.

Flat Tire:

If you encounter a flat tire while driving, you can temporarily seal the puncture and reinflate the tire using a tire mobility kit. This kit is typically stored under the lining of the luggage area in your vehicle.

Instructions for using the tire mobility kit:

    Open the tailgate or trunk of your vehicle.
    Lift up the lining of the luggage area to access the tire mobility kit.
    Follow the instructions provided with the tire mobility kit to seal the puncture in the tire.
    After using the kit, make sure to securely put it back in its original location.
    Contact AnyCompany or an appropriate service for assistance with disposing of and replacing the used sealant bottle.

Please note that the tire mobility kit is a temporary solution and is designed to allow you to drive for a maximum of 10 minutes or 8 km (whichever comes first) at a maximum speed of 80 km/h. It is advisable to replace the punctured tire or have it repaired by a professional as soon as possible."""

##### Agora, transmita o trecho inteiro para o modelo com a pergunta.

In [ ]:
question = "How can I fix a flat tire on my AnyCompany AC8?"
prompt_data = f"""Answer the question based only on the information provided between ## and give step by step guide in 5 steps with conclusion.
#
{context}
#

Question: {question}
Answer:"""

### Tarefa 3.4: Invocar o modelo via boto3 para gerar a resposta

In [ ]:
import json
import time
from botocore.exceptions import ClientError

def invoke_nova_lite_with_retry(prompt_data, max_retries=5, initial_delay=10):
    # Format the prompt for Nova Lite
    body = json.dumps(format_for_nova_lite(prompt_data))
    modelId = "amazon.nova-lite-v1:0"  
    accept = "application/json"
    contentType = "application/json"

    for attempt in range(max_retries):
        try:
            response = bedrock_client.invoke_model(
                body=body, 
                modelId=modelId, 
                accept=accept, 
                contentType=contentType
            )
            response_body = json.loads(response.get("body").read())
            return parse_nova_lite_response(response_body)

        except ClientError as error:
            error_code = error.response['Error']['Code']
            
            if error_code == 'AccessDeniedException':
                print(f"\x1b[41m{error.response['Error']['Message']}\\n\
                \nTo troubleshoot this issue please refer to the following resources:\\n\
                \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\\n\
                \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
                raise
            
            elif error_code in ['ThrottlingException', 'ServiceUnavailableException', 'ModelStreamLimitExceededException']:
                if attempt < max_retries - 1:
                    delay = initial_delay * (2 ** attempt)  # Exponential backoff
                    print(f"Service capacity reached. Retrying in {delay} seconds...")
                    print(f"Error: {error}")
                    time.sleep(delay)
                    continue
                else:
                    print(f"Max retries ({max_retries}) reached. Last error: {error}")
                    raise
            else:
                print(f"Unhandled error occurred: {error}")
                raise
                
        except Exception as e:
            if attempt < max_retries - 1:
                delay = initial_delay * (2 ** attempt)
                print(f"Unexpected error: {e}")
                print(f"Retrying in {delay} seconds...")
                time.sleep(delay)
                continue
            else:
                print(f"Failed after {max_retries} attempts. Last error: {e}")
                raise

# Usage
try:
    answer = invoke_nova_lite_with_retry(prompt_data)
    print(answer.strip())
except Exception as e:
    print(f"Final error: {e}")


Como o modelo demora um pouco para compreender o contexto e gerar uma resposta relevante para você, isso pode resultar em uma experiência do usuário ruim, pois é necessário aguardar a resposta por alguns segundos.

O Bedrock também é compatível com o recurso de transmissão em que o serviço gera saída conforme o modelo gera tokens. Aqui, é possível ver um exemplo de como você pode implementar isso.

In [ ]:
from IPython.display import display_markdown,Markdown,clear_output

In [ ]:
import json
import time
from botocore.exceptions import ClientError

def invoke_nova_lite_stream_with_retry(prompt_data, max_retries=5, initial_delay=10):
    body = json.dumps(format_for_nova_lite(prompt_data))
    modelId = "amazon.nova-lite-v1:0"
    accept = "application/json"
    contentType = "application/json"
    
    print(f"Using model: {modelId}")

    for attempt in range(max_retries):
        try:
            response = bedrock_client.invoke_model_with_response_stream(
                body=body, 
                modelId=modelId, 
                accept=accept, 
                contentType=contentType
            )
            
            stream = response.get('body')
            output = []
            i = 1
            
            if stream:
                for event in stream:
                    chunk = event.get('chunk')
                    if chunk:
                        chunk_obj = json.loads(chunk.get('bytes').decode())
                        # Extract text from Nova Lite's response format
                        if 'contentBlockDelta' in chunk_obj and 'delta' in chunk_obj['contentBlockDelta']:
                            text = chunk_obj['contentBlockDelta']['delta'].get('text', '')
                            clear_output(wait=True)
                            output.append(text)
                            display_markdown(Markdown(''.join(output)))
                            i += 1
                return ''.join(output)  # Return the complete output
            else:
                raise Exception("No stream data received")

        except ClientError as error:
            error_code = error.response['Error']['Code']
            
            if error_code == 'AccessDeniedException':
                print(f"\x1b[41m{error.response['Error']['Message']}\\n\
                \nTo troubleshoot this issue please refer to the following resources:\\n\
                \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\\n\
                \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
                raise
            
            elif error_code in ['ThrottlingException', 'ServiceUnavailableException', 'ModelStreamLimitExceededException']:
                if attempt < max_retries - 1:
                    delay = initial_delay * (2 ** attempt)  # Exponential backoff
                    print(f"Service capacity reached. Retrying in {delay} seconds...")
                    print(f"Error: {error}")
                    time.sleep(delay)
                    continue
                else:
                    print(f"Max retries ({max_retries}) reached. Last error: {error}")
                    raise
            else:
                print(f"Unhandled error occurred: {error}")
                raise
                
        except Exception as e:
            if attempt < max_retries - 1:
                delay = initial_delay * (2 ** attempt)
                print(f"Unexpected error: {e}")
                print(f"Retrying in {delay} seconds...")
                time.sleep(delay)
                continue
            else:
                print(f"Failed after {max_retries} attempts. Last error: {e}")
                raise

# Usage
try:
    # You can adjust retry parameters here
    result = invoke_nova_lite_stream_with_retry(
        prompt_data,
        max_retries=5,
        initial_delay=10
    )

except Exception as e:
    print(f"Final error: {e}")


A resposta fornece instruções passo a passo resumidas sobre como trocar os pneus. 

Você aprendeu a usar a geração aumentada de recuperação (RAG) ou o processo de aumento para gerar uma resposta com curadoria adaptada ao contexto específico e às informações fornecidas.

### Experimente você mesmo
- Altere os prompts para seu caso de uso específico e avalie o resultado de diferentes modelos.
- Teste o comprimento do token para entender a latência e a responsividade do serviço.
- Aplique diferentes princípios de engenharia de prompts para gerar resultados melhores.

### Limpeza

Você concluiu este caderno. Passe para a próxima parte do laboratório da seguinte forma:

- Feche este arquivo de caderno e continue com a **Tarefa 4**.